In [82]:
import numpy as nm
import pandas as pd
import csv
import wrds

In [83]:
f = open('2010_2020_daily_CRSP.csv')
csv_f = csv.reader(f)

In [84]:
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [85]:
db.get_table('djones', 'djdaily', columns=['date', 'dji'], obs=10)

,date,dji
0,1896-05-26,40.94
1,1896-05-27,40.58
2,1896-05-28,40.20
3,1896-05-29,40.63
4,1896-06-01,40.60
5,1896-06-02,40.04
6,1896-06-03,39.77
7,1896-06-04,39.94
8,1896-06-05,40.32
9,1896-06-08,39.81


In [18]:
data = db.get_row_count('djones', 'djdaily')
data

28003

In [43]:
#db.list_tables(library='crsp')
#db.describe_table(library="crsp", table="stocknames")
db.describe_table(library="crsp", table='saz_ds_dly')

Approximately 98216170 rows in crsp.saz_ds_dly.


,name,nullable,type
0,kypermno,True,DOUBLE_PRECISION
1,caldt,True,DATE
2,bidlo,True,DOUBLE_PRECISION
3,askhi,True,DOUBLE_PRECISION
4,bid,True,DOUBLE_PRECISION
5,ask,True,DOUBLE_PRECISION
6,numtrd,True,DOUBLE_PRECISION
7,openprc,True,DOUBLE_PRECISION


In [29]:
help(db.get_table)

Help on method get_table in module wrds.sql:

get_table(library, table, obs=-1, offset=0, columns=None, coerce_float=None, index_col=None, date_cols=None) method of wrds.sql.Connection instance
    Creates a data frame from an entire table in the database.
    
    :param sql: SQL code in string object.
    :param library: Postgres schema name.
    
    :param obs: (optional) int, default: -1
        Specifies the number of observations to pull from the table.
        An integer less than 0 will return the entire table.
    :param offset: (optional) int, default: 0
        Specifies the starting point for the query.
        An offset of 0 will start selecting from the beginning.
    :param columns: (optional) list or tuple, default: None
        Specifies the columns to be included in the output data frame.
    :param coerce_float: (optional) boolean, default: True
        Attempt to convert values to non-string, non-numeric objects
        to floating point. Can result in loss of prec

In [87]:
stocknames = db.get_table(library='crsp', table='stocknames', obs=5)
>>> stocknames.head()

,permno,permco,namedt,nameenddt,cusip,ncusip,ticker,comnam,hexcd,exchcd,siccd,shrcd,shrcls,st_date,end_date,namedum
0,10000.0,7952.0,1986-01-07,1987-06-11,68391610,68391610,OMFGA,OPTIMUM MANUFACTURING INC,3.0,3.0,3990.0,10.0,A,1986-01-31,1987-06-30,2.0
1,10001.0,7953.0,1986-01-09,1993-11-21,36720410,39040610,GFGC,GREAT FALLS GAS CO,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
2,10001.0,7953.0,1993-11-22,2008-02-04,36720410,29274A10,EWST,ENERGY WEST INC,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
3,10001.0,7953.0,2008-02-05,2009-08-03,36720410,29274A20,EWST,ENERGY WEST INC,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
4,10001.0,7953.0,2009-08-04,2009-12-17,36720410,29269V10,EGAS,ENERGY INC,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0


In [73]:
stocknames = db.get_table(library='crsp', table="stocknames", columns=['permno','permco','ticker','comnam'], obs=10)
>>> stocknames.head()

OperationalError: (psycopg2.OperationalError) SSL SYSCALL error: No error (0x00000000/0)

(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [88]:
stocknames = db.get_table(library="crsp", table='saz_ds_dly', columns=['KYPERMNO','CALDT','BIDLO','ASKHI','BID','ASK'], obs=10)


In [89]:
stocknames.head()

,kypermno,caldt,bidlo,askhi,bid,ask
0,10000.0,1986-01-07,2.375,2.750,None,None
1,10000.0,1986-01-08,2.375,2.625,None,None
2,10000.0,1986-01-09,2.375,2.625,None,None
3,10000.0,1986-01-10,2.375,2.625,None,None
4,10000.0,1986-01-13,2.500,2.750,None,None


In [90]:
stocknames = db.get_table(library="crsp", table='saz_ds_dly', obs=10)
stocknames.head()

,kypermno,caldt,bidlo,askhi,bid,ask,numtrd,openprc
0,10000.0,1986-01-07,2.375,2.750,None,None,None,None
1,10000.0,1986-01-08,2.375,2.625,None,None,None,None
2,10000.0,1986-01-09,2.375,2.625,None,None,None,None
3,10000.0,1986-01-10,2.375,2.625,None,None,None,None
4,10000.0,1986-01-13,2.500,2.750,None,None,None,None


In [131]:
dly1 = db.get_table(library="crsp", table='saz_ds_dly', obs=1000)
dly2 = db.get_table(library="crsp", table='saz_dp_dly',obs=1000)
dly3 = pd.merge(dly1, dly2, on=["kypermno", "caldt"])
dly3.head()

,kypermno,caldt,bidlo,askhi,bid,ask,numtrd,openprc,prc,ret,retx,tcap,vol
0,10000.0,1986-01-07,2.375,2.750,NaN,NaN,NaN,None,-2.5625,NaN,NaN,9430.0,1000.0
1,10000.0,1986-01-08,2.375,2.625,NaN,NaN,NaN,None,-2.5000,-0.02439,-0.02439,9200.0,12800.0
2,10000.0,1986-01-09,2.375,2.625,NaN,NaN,NaN,None,-2.5000,0.00000,0.00000,9200.0,1400.0
3,10000.0,1986-01-10,2.375,2.625,NaN,NaN,NaN,None,-2.5000,0.00000,0.00000,9200.0,8500.0
4,10000.0,1986-01-13,2.500,2.750,NaN,NaN,NaN,None,-2.6250,0.05000,0.05000,9660.0,5450.0


In [132]:
dly3.drop(columns=['prc','ret','retx','tcap'])

,kypermno,caldt,bidlo,askhi,bid,ask,numtrd,openprc,vol
0,10000.0,1986-01-07,2.375,2.750,NaN,NaN,NaN,None,1000.0
1,10000.0,1986-01-08,2.375,2.625,NaN,NaN,NaN,None,12800.0
2,10000.0,1986-01-09,2.375,2.625,NaN,NaN,NaN,None,1400.0
3,10000.0,1986-01-10,2.375,2.625,NaN,NaN,NaN,None,8500.0
4,10000.0,1986-01-13,2.500,2.750,NaN,NaN,NaN,None,5450.0
...,...,...,...,...,...,...,...,...,...
995,10001.0,1988-07-12,6.250,6.250,6.25,6.75,1.0,None,300.0
996,10001.0,1988-07-13,6.250,6.750,6.25,6.75,0.0,None,0.0
997,10001.0,1988-07-14,6.250,6.750,6.25,6.75,0.0,None,0.0
998,10001.0,1988-07-15,6.250,6.750,6.25,6.75,0.0,None,0.0


In [133]:
for row in dly3.itertuples():
    permno = row.kypermno*1
    dly3.to_csv(str(permno)+'.csv', index = False)

In [150]:
df['kypermno'].unique()

array([10000., 10001.])

In [147]:
df=dly3

In [151]:
for permco in df['kypermno'].unique():
    sub_df = df[df['kypermno'] == permco]
    sub_df.to_csv(f'{permco}.csv', index=False)

In [152]:
df['prc'].plot()